In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import xgboost

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor 
from sklearn.inspection import plot_partial_dependence, partial_dependence

from sklearn.linear_model import Lasso, LinearRegression
 
import shap
from scipy.linalg import toeplitz

import seaborn as sns
sns.set(style="whitegrid")

# DGP

In [ ]:
np.random.seed(1)

n=10000
k = 6
rho = 0.6
cova = rho**toeplitz(range(6), range(6))

X =  np.random.multivariate_normal(mean=[0]*k, cov=cova, size=n)  ### TFIDF
beta = np.random.uniform(low=-1.0, high=1.0, size=(k,1))
beta = beta/np.abs(beta).sum()
eps = np.random.normal(loc=0.0, scale=0.1, size=(n,1))

f = [lambda x:x, lambda x:x**2, lambda x: np.sin(x), lambda x: np.exp(x), lambda x: np.abs(np.log(abs(x))),
     lambda x: x*(x>0)]

#y = beta[0] * X[:,0] + beta + eps

y = np.zeros((n,1))
for j in range(k):
    
    y +=  (beta[j] * f[j](X[:,j])).reshape((n,1))
    
        
y = y + eps


In [ ]:
beta

In [ ]:
plt.hist(y)
plt.show()

In [ ]:
nrows, ncols = 3,2
fig, ax = plt.subplots(nrows=nrows,ncols=ncols, figsize=(20,12))

line=0
col =0
pos = 0


for var in range(k):
    
    ax[line,col].plot(X[:,var], beta[var] * f[var](X[:,var]), ".")
    ax.flat[pos].label_outer()
    
    if ((col+1)%ncols==0):
        line+=1
        col=0

    else:
        col+=1

    pos+=1

    
plt.rc('text', usetex=True)
for i,ax in enumerate(fig.axes):
    plt.sca(ax)
    plt.xlabel(r"$X_%s$"%(i+1), fontsize=16)
    plt.ylabel(r"$\beta_%sf_%s(X_%s)$"%(i+1,i+1,i+1), fontsize=16)
    
    
plt.savefig("C:/Users/hgill/Documents/Etudes/DABSA/Redaction/pics/betaf.png")

# Computing Shap values with the Gradient Boosting algorithm

In [ ]:
%%time
model = xgboost.train(params={"learning_rate": 0.01}, dtrain=xgboost.DMatrix(X, label=y), num_boost_round=1000)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)


In [ ]:
nrows, ncols = 3,2
fig, ax = plt.subplots(nrows=nrows,ncols=ncols, figsize=(20,12))

line=0
col =0
pos = 0


for var in range(k):
    
    ax[line,col].plot(X[:,var], shap_values[:,var], ".")
    ax.flat[pos].label_outer()
    
    if ((col+1)%ncols==0):
        line+=1
        col=0

    else:
        col+=1

    pos+=1

    
plt.rc('text', usetex=True)
for i,ax in enumerate(fig.axes):
    plt.sca(ax)
    plt.xlabel(r"$X_%s$"%(i+1), fontsize=16)
    plt.ylabel(r"$\hat{\beta_%s}\hat{f}_%s(X_%s)$"%(i+1,i+1,i+1), fontsize=16)

plt.savefig("C:/Users/hgill/Documents/Etudes/DABSA/Redaction/pics/betaf_hat.png")

In [ ]:
model.predict(xgboost.DMatrix(X)).mean()

In [ ]:
linear = LinearRegression()

linear.fit(shap_values.sum(axis=1).reshape(-1,1), y)

beta_hat = linear.coef_.ravel()[0]
intercept = linear.intercept_[0]

plt.figure(figsize=(10,4))
plt.plot(shap_values.sum(axis=1).reshape(-1,1), y, ".")
plt.xlabel(r"$\displaystyle \sum_{j=1}^{6}\hat{\phi_j}$", fontsize=16)
plt.ylabel(r"$y$", fontsize=16)

plt.text(8,0, r" $\hat{y} = %s + %s \displaystyle \sum_{j=1}^{6}\hat{\phi_j}$"%(round(intercept,2), round(beta_hat,2)), fontsize=16)
plt.savefig("C:/Users/hgill/Documents/Etudes/DABSA/Redaction/pics/ySumPhi_hat.png")
plt.show()
